In [1]:
import os
from r2r import R2RClient
from scraper import Scraper
from splitter import Splitter
from io_helper import IOHelper
from server import ServerHelper
from prompt import PromptHelper

client = R2RClient(os.getenv("R2R_HOSTNAME", "http://localhost:7272"))
scraper = Scraper()
splitter = Splitter()
io_helper = IOHelper()
server = ServerHelper(client)
prompt = PromptHelper(client)

2024-10-20 15:55:20,231 - WARNING - langchain_community.utils.user_agent - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
server.health()

{'response': 'ok'}

In [20]:
import yaml
from pathlib import Path

def load_prompt(directory_path):
    if not directory_path:
        raise ValueError("No directory path provided")
    
    if not directory_path.is_dir():
        error_msg = f"The specified path is not a directory: {directory_path}"
        raise ValueError(error_msg)

    for yaml_file in directory_path.glob("*.yaml"):
        try:
            with open(yaml_file, "r") as file:
                data = yaml.safe_load(file)
                if not data:
                    print(f"Warning: YAML file {yaml_file} is empty")
                    continue
                if not isinstance(data, dict):
                    raise ValueError(f"Invalid format in YAML file {yaml_file}")
                
                for name, prompt_data in data.items():
                    print(f'{name}, {prompt_data["template"]}, {prompt_data.get("input_types", {})}')
                    
        except yaml.YAMLError as e:
            error_msg = f"Error loading prompts from YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)
        except KeyError as e:
            error_msg = f"Missing key in YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)
        except Exception as e:
            error_msg = f"Unexpected error loading YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)

In [21]:
load_prompt(Path("my_templates"))

custom_rag_template, ## Task: You are an AI assistant specialized in providing accurate and context-based responses. Your task is to answer user queries using only the information from the provided context. Your responses should be factual, comprehensive, and directly reference the given context.
## Context: {context}
## Query: {query}
## Instructions: 1. Analyze the query and the provided context carefully. 2. Formulate a response based solely on the information in the context. 3. Use specific references (e.g., [1], [2]) to cite relevant parts of the context in your response. 4. Ensure your answer is as comprehensive as possible within the constraints of the given context. 5. If the query cannot be fully answered from the context:
  a. Provide partial information if available, clearly stating what aspects you can address.
  b. Indicate which parts of the query cannot be answered based on the given context.
6. If the query is entirely outside the scope of the provided context, state: "

In [9]:
from r2r import R2RPromptProvider

In [5]:
files = list(io_helper.iterate_over_files("files"))
ingestion_resp = server.ingest_files(files)

Ingested: files\2311.09476v2.pdf ...


In [6]:
from r2r import R2RClient

client = R2RClient()

template = client.get_all_prompts()['results']['prompts']['default_rag']['template']
#print(client.get_all_prompts()['results']['prompts']['rag_context']['template'])

In [9]:
chunks = server.document_chunks(doc_id)

In [7]:
rag_rsp = server.rag(
    query="What is ARES?",
    task_prompt_override=template
)

In [4]:
print(rag_rsp['completion']['choices'][0]['message']['content'])

(0, 1)


In [3]:
from prompt import PromptHelper

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
